In [ ]:
## reference

# https://medium.com/analytics-vidhya/tf-gradienttape-explained-for-keras-users-cc3f06276f22

In [10]:
import tensorflow as tf
import numpy as np
import random

In [2]:
## test tf.GradientTape

x = tf.constant(5.0)

with tf.GradientTape() as tape:
    tape.watch(x)
    y = x ** 3

print(tape.gradient(y, x).numpy())
print(3 * x ** 2)

75.0
tf.Tensor(75.0, shape=(), dtype=float32)


In [3]:
x = tf.Variable(6.0, trainable = True)

with tf.GradientTape() as tape:
    y = x ** 3

print(tape.gradient(y, x).numpy())

108.0


In [6]:
x = tf.Variable(6.0, trainable = False)

with tf.GradientTape() as tape:
    y = x ** 3

# print(tape.gradient(y, x).numpy())
# AttributeError: 'NoneType' object has no attribute 'numpy'

In [9]:
## higher order derivatives

x = tf.Variable(3.0, trainable = True)

with tf.GradientTape() as tape1:
    with tf.GradientTape() as tape2:
        y = x ** 3
    order_1 = tape2.gradient(y, x)
order_2 = tape1.gradient(order_1, x)

print(order_2.numpy())
print(3 * x ** 2)
# differentiate x ** 3 two times with respect to x
print(3 * 2 * x)

18.0
tf.Tensor(27.0, shape=(), dtype=float32)
tf.Tensor(18.0, shape=(), dtype=float32)


In [14]:
## persistent

a = tf.Variable(6.0, trainable = True)
b = tf.Variable(2.0, trainable = True)

with tf.GradientTape() as tape:
    y1 = a ** 2
    y2 = b ** 3

print(tape.gradient(y1, a).numpy())
# below produces RuntimeError: GradientTape.gradient can only be called once on non-persistent tapes.
# print(tape.gradient(y2, b).numpy())

a = tf.Variable(6.0, trainable = True)
b = tf.Variable(2.0, trainable = True)

with tf.GradientTape(persistent = True) as tape:
    y1 = a ** 2
    y2 = b ** 3

# take gradient with respect to a
print(tape.gradient(y1, a).numpy())
# take gradient with respect to b
print(tape.gradient(y2, b).numpy())

12.0
12.0
12.0


In [28]:
## linear regression

def loss(y_true, y_pred):
    """
    Loss function
    Compute L1 loss
    """
    return tf.abs(y_true - y_pred)

# training data
x_train = np.asarray([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
# y = 10x + 5
y_train = np.asarray([10 * i + 5 for i in x_train])
lr = 0.001
epochs = 1000

# trainable variables which are trained in epochs
a = tf.Variable(random.random(), trainable = True)
b = tf.Variable(random.random(), trainable = True)

def step(x_true, y_true):
    with tf.GradientTape(persistent = True) as tape:
        # forward pass
        y_pred = a * x_true + b
        # calculate loss
        reg_loss = loss(y_true, y_pred)

    # take gradient with respect to a and b
    a_gradient, b_gradient = tape.gradient(reg_loss, (a, b))
    # Update variables
    a.assign_sub(a_gradient * lr)
    b.assign_sub(b_gradient * lr)

# training
for _ in range(epochs):
    step(x_train, y_train)

print(f'y = {a.numpy()} x + {b.numpy()}')

y = 9.998836517333984 x + 4.990261077880859
